In [1]:
import scanpy as sc 
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import bbknn
from scipy import stats
from pyecharts import options as opts
from pyecharts.charts import Sankey
import pandas as pd
import matplotlib
matplotlib.use('TkAgg') # 指定
%matplotlib inline
# sc.settings.set_figure_params(figsize=(5,4)) # 设置全局产生的图片尺寸
sc.settings.verbosity = 0 # 设置全局的waring数量，我这里直接不要求显示
sc.settings.set_figure_params(dpi=100, facecolor='white', fontsize=7, format='jpg')

In [10]:
adata = sc.read_h5ad('../data/v4-sub-fib.h5ad')
val = []
for x in list(adata.obs['sample']):
    if x.startswith('AF'):
        val.append('AF')
    else:
        val.append('NAF')
adata.obs['sample_group'] = np.array(val)

In [11]:
adata

AnnData object with n_obs × n_vars = 27876 × 3000
    obs: 'sample', 'n_genes', 'n_genes_by_counts', 'total_counts', 'total_counts_mt', 'pct_counts_mt', 'leiden', 'leiden_poly', 'sample_group'
    var: 'gene_ids', 'feature_types', 'n_cells', 'mt', 'n_cells_by_counts', 'mean_counts', 'pct_dropout_by_counts', 'total_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm', 'mean', 'std'
    uns: 'dendrogram_leiden', 'hvg', 'leiden', 'leiden_colors', 'leiden_poly_colors', 'log1p', 'neighbors', 'pca', 'rank_genes_groups', 'sample_colors', 'umap'
    obsm: 'X_pca', 'X_umap'
    varm: 'PCs'
    obsp: 'connectivities', 'distances'

In [12]:
# 选择你想细分的类，有可能是leiden_ploy这个key内存了你的大类，可以两个都试试如果报错
# adata = adata[adata.obs['leiden_poly'].isin(['Fibroblasts'])] 
n_af = adata[adata.obs['sample_group'].isin(['AF'])].shape[0] 
n_naf = adata[adata.obs['sample_group'].isin(['NAF'])].shape[0] 
print('cells: ', adata.obs.shape[0])
print('AF ', n_af)
print('NAF: ', n_naf)

cells:  27876
AF  16053
NAF:  11823


In [13]:
df = adata.obs.groupby(by=['leiden_poly','sample_group']).count().loc[:,'sample']

In [14]:
df = pd.DataFrame(df)

In [15]:
df.index

MultiIndex([('Fib1',  'AF'),
            ('Fib1', 'NAF'),
            ('Fib2',  'AF'),
            ('Fib2', 'NAF'),
            ('Fib3',  'AF'),
            ('Fib3', 'NAF')],
           names=['leiden_poly', 'sample_group'])

In [16]:
node = []
links = []
for val in list(df.index):
    dic = {}
    dic['source'] = val[0]
    dic['target'] = val[1]
    dic['value'] = df.loc[val[0],:].loc[val[1],'sample']
    
    links.append(dic)

for val in df.index:
    node += list(val)
nodes = []
node = np.unique(node)
for val in node:
    nodes.append({'name':val})

In [17]:
df

sample
leiden_poly sample_group        
Fib1        AF             12230
            NAF             7282
Fib2        AF              3574
            NAF             3183
Fib3        AF               249
            NAF             1358

In [22]:
nodes=[{'name': 'AF'},
  {'name': 'Fib1'},
  {'name': 'Fib2'},
  {'name': 'Fib3'},
  {'name': 'NAF'}]
links= [{'source': 'Fib1', 'target': 'AF', 'value': 12230},
  {'source': 'Fib1', 'target': 'NAF', 'value': 7282},
  {'source': 'Fib2', 'target': 'AF', 'value': 3574},
  {'source': 'Fib2', 'target': 'NAF', 'value': 3183},
  {'source': 'Fib3', 'target': 'AF', 'value': 249},
  {'source': 'Fib3', 'target': 'NAF', 'value': 1358}]

In [23]:
pic=(
    Sankey().add(
        '',#图例名称
        nodes,#传入节点数据
        links,#传入边和流量数据
        #设置透明度、弯曲度、颜色
        linestyle_opt=opts.LineStyleOpts(opacity=0.3,curve=0.5,color='source'),
        #标签显示位置
        label_opts=opts.LabelOpts(),
        #节点之间的距离
        node_gap=30,
    )
    .set_global_opts()
)
pic.render('test.html')

'/Users/ckw/project/knee/analysis/notebook/test.html'

In [10]:
links

[{'source': 'fib_2', 'target': 'AF', 'value': 2587},
 {'source': 'fib_2', 'target': 'NAF', 'value': 3035},
 {'source': 'fib_APOD', 'target': 'AF', 'value': 254},
 {'source': 'fib_APOD', 'target': 'NAF', 'value': 1320},
 {'source': 'fib_CD55', 'target': 'AF', 'value': 13304},
 {'source': 'fib_CD55', 'target': 'NAF', 'value': 7633}]